In [1]:
import cv2 as cv
import numpy as np
import scipy
import pickle
import random
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans,MiniBatchKMeans
from sklearn.decomposition import LatentDirichletAllocation
from scipy.spatial.distance import euclidean,cosine



In [2]:
directory='./101_ObjectCategories'
img_dirs=os.listdir(directory)[:10]

In [3]:
#finding descriptors
desc=[]
X=[]
sift = cv.xfeatures2d.SIFT_create()
final_images=[]
for img_dir in img_dirs:
    images = os.listdir(directory+'/'+img_dir)
    for img_name in images:
        img_path=os.path.join(directory,img_dir,img_name)
        img=cv.imread(img_path)
        kps=sift.detect(img)
        kp1, des1 = sift.compute(img, kps)
        try:
            if (des1.all!=None):
                final_images.append(img_path)
                desc.append([])
                desc[-1].append(des1)
                for d in des1:
                    X.append(d)
        except:
            pass

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-ffy7zady\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 164581680 bytes in function 'cv::OutOfMemoryError'


In [23]:
output = open('descriptor.pkl', 'wb')
pickle.dump(desc,output,pickle.HIGHEST_PROTOCOL) 

In [24]:
out = open('images.pkl', 'wb')
pickle.dump(final_images,out,pickle.HIGHEST_PROTOCOL)

In [25]:
#clustering of key points to make dictionary using k means
dictionary_size=2000
kmeans = KMeans(n_clusters=dictionary_size, init='k-means++', max_iter=300, n_init=10, random_state=0).fit(X)
centers=kmeans.cluster_centers_
print("clustered")

clustered


In [26]:
#flann matcher to match key points to words in dictionary
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50) 

flann = cv.FlannBasedMatcher(index_params,search_params)
arr=[]
for i in range(len(desc)):
    matches = flann.match(np.asarray(desc[i][0],np.float32),np.asarray(centers,np.float32))
    #for j in range(len(matches)):
        #arr[-1].append(matches[j][0].trainIdx)
    arr.append([matches[j].trainIdx for j in range(len(matches))])


In [27]:
'''
#creating a histogram
doc=[]
for i in range(len(arr)): 
    hist=[0]*(dictionary_size)
    for j in range(0,dictionary_size):
        l = len((np.where(np.array(arr[i]) == j))[0])
        hist[j] = l
    doc.append(hist)
print("histogram with frequency of words in an image done (method 1)")
'''

'\n#creating a histogram\ndoc=[]\nfor i in range(len(arr)): \n    hist=[0]*(dictionary_size)\n    for j in range(0,dictionary_size):\n        l = len((np.where(np.array(arr[i]) == j))[0])\n        hist[j] = l\n    doc.append(hist)\nprint("histogram with frequency of words in an image done (method 1)")\n'

In [28]:
doc_word=[]
for i in range(len(arr)):
    hist=[0]*dictionary_size
    c={}
    for j in range(len(arr[i])):
        if (arr[i][j] in c):
            c[arr[i][j]]+=1
        else:
            c[arr[i][j]]=1
    for k in c:
        hist[k]=c[k]
    doc_word.append(hist)
print("histogram with frequency of words in an image done (method 2)")       
    

histogram with frequency of words in an image done (method 2)


In [29]:
print(doc_word)

[[2, 8, 0, 6, 5, 8, 16, 6, 3, 2, 11, 2, 2, 3, 22, 13, 13, 2, 1, 0, 5, 3, 0, 4, 11, 2, 0, 2, 21, 2, 8, 8, 1, 2, 2, 9, 0, 0, 6, 4, 1, 13, 3, 5, 8, 4, 1, 12, 4, 8, 11, 9, 7, 5, 1, 0, 4, 12, 2, 1, 1, 2, 3, 13, 6, 2, 2, 1, 4, 3, 4, 2, 7, 5, 1, 4, 3, 3, 7, 3, 3, 1, 4, 2, 0, 5, 12, 5, 4, 7, 2, 5, 19, 7, 0, 1, 1, 4, 1, 2], [11, 3, 10, 12, 9, 2, 3, 9, 20, 3, 3, 9, 25, 9, 0, 3, 1, 6, 16, 6, 4, 7, 2, 5, 3, 8, 22, 1, 0, 7, 3, 4, 7, 3, 4, 9, 4, 6, 4, 12, 31, 1, 7, 12, 1, 9, 5, 6, 7, 26, 1, 5, 4, 5, 6, 10, 12, 2, 19, 1, 2, 15, 3, 12, 10, 17, 9, 3, 10, 3, 48, 3, 12, 12, 4, 8, 31, 13, 7, 4, 7, 8, 5, 2, 7, 13, 13, 3, 6, 5, 16, 15, 1, 3, 13, 25, 11, 35, 6, 6], [6, 3, 16, 0, 1, 5, 1, 2, 7, 0, 3, 6, 3, 2, 2, 1, 2, 1, 4, 2, 3, 0, 2, 1, 4, 8, 21, 2, 27, 1, 3, 1, 5, 4, 2, 8, 2, 6, 3, 17, 19, 3, 4, 2, 3, 4, 2, 5, 8, 3, 4, 4, 1, 4, 2, 1, 9, 2, 6, 2, 3, 1, 3, 4, 11, 8, 14, 6, 6, 9, 4, 3, 10, 5, 2, 3, 6, 2, 4, 4, 2, 3, 7, 2, 2, 4, 3, 3, 0, 5, 12, 2, 2, 4, 9, 15, 8, 5, 8, 7], [11, 4, 14, 2, 2, 4, 3, 6, 21, 1, 3, 

In [30]:
# topic modelling 
ld = LatentDirichletAllocation(n_components=6,learning_method='online',max_iter=100)
ld.fit(doc_word)
topic_dist=ld.transform(doc_word)


In [31]:
topic_dist


array([[3.44684349e-04, 3.45086825e-04, 2.20697271e-01, 3.44830804e-04,
        3.44925525e-04, 7.77923202e-01],
       [6.50488752e-02, 1.93219700e-04, 6.34286396e-03, 7.69079018e-01,
        1.59142959e-01, 1.93064041e-04],
       [3.45818706e-04, 3.45992498e-04, 3.45912539e-04, 3.45810200e-01,
        4.49203670e-01, 2.03948406e-01],
       [2.04544164e-01, 2.83646990e-04, 1.65706145e-01, 2.83475521e-04,
        6.28898946e-01, 2.83622377e-04],
       [2.34904373e-04, 2.34829571e-04, 2.34840888e-04, 2.34507692e-04,
        2.34927689e-04, 9.98825990e-01],
       [4.53286315e-04, 4.51734921e-04, 4.51067656e-04, 9.97738937e-01,
        4.52204094e-04, 4.52770360e-04],
       [9.62216239e-04, 9.63523068e-04, 6.18474537e-02, 9.61439645e-04,
        9.34302070e-01, 9.63297016e-04],
       [6.27396567e-01, 3.56202258e-04, 2.36908819e-01, 3.56106681e-04,
        1.34625833e-01, 3.56472276e-04],
       [9.69668533e-02, 1.88247640e-04, 3.39551117e-02, 5.98622631e-01,
        4.38603140e-02, 

In [32]:
#querying an image 
query_image_path='D:/Chirag/Stuff/Projects/Sem 5/Mini-Project/Data/Caltech/101_ObjectCategories/crab/image_0001.png'
query_img=cv.imread(query_image_path)
query_img_kps=sift.detect(query_img)
query_kp, query_des = sift.compute(query_img,query_img_kps)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50) 

flann = cv.FlannBasedMatcher(index_params,search_params)
query_arr=[]
matches = flann.match(np.asarray(query_des,np.float32),np.asarray(centers,np.float32))
query_arr.append([matches[j].trainIdx for j in range(len(matches))])

dictionary_size=100
q={}
query_hist=[0]*dictionary_size
for j in range(len(arr[0])):
    if (arr[0][j] in q):
        q[arr[0][j]]+=1
    else:
        q[arr[0][j]]=1
for k in q:
    query_hist[k]=q[k]

query_hist=np.array(query_hist).reshape(1,-1)

query_topic = ld.transform(query_hist)

euc=[]
for i in range(len(topic_dist)):
    euc.append([final_images[i],euclidean(query_topic[0],topic_dist[i])])
    
img_sorted = sorted(euc, key=lambda x: x[1])


In [33]:
print(img_sorted)

[['image_0001.jpg', 0.0], ['image_0061.jpg', 0.04091471012856025], ['image_0031.jpg', 0.11711316358381131], ['image_0023.jpg', 0.21298376624866153], ['image_0047.jpg', 0.2507840630875001], ['image_0035.jpg', 0.27672439282912814], ['image_0053.jpg', 0.29139135902629687], ['image_0057.jpg', 0.2971957421632785], ['image_0005.jpg', 0.3120925715084749], ['image_0036.jpg', 0.3420581204256247], ['image_0026.jpg', 0.4342823055152857], ['image_0070.jpg', 0.47087315197915364], ['image_0014.jpg', 0.47436245992550824], ['image_0064.jpg', 0.5005560951910909], ['image_0056.jpg', 0.5197954266048901], ['image_0010.jpg', 0.5221219181302296], ['image_0072.jpg', 0.555431005969552], ['image_0027.jpg', 0.5686754715097638], ['image_0029.jpg', 0.5854179471202436], ['image_0059.jpg', 0.6475244544643106], ['image_0032.jpg', 0.6507602422622187], ['image_0060.jpg', 0.669653369510598], ['image_0016.jpg', 0.6860380303557359], ['image_0037.jpg', 0.6911002291610703], ['image_0050.jpg', 0.7332402925662522], ['image_0

In [4]:
for i in range(10):
    out = cv2.imread(img_sorted[i][0],1)
    cv2.imshow(str(i),out)

NameError: name 'cv2' is not defined

In [ ]:
_